In [3]:
import numpy as np
import sympy as sy
from sympy import *
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats as sts
import scipy. integrate as intr
import seaborn
from autograd import numpy as anp
from autograd import grad
from autograd import elementwise_grad
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pprint import pprint
max_line_width=np.inf
from scipy import linalg 

In [4]:
class simplex_algo:
    def __init__(self, c, A, b):
        self.c = c
        self.A = A
        self.b = b
        self.n = len(c)
        if np.any(self.A@np.zeros((self.n, 1))>self.b):
            raise ValueError('Not feasible')
    
        self.m,self.n = self.A.shape
        ind =  np.concatenate((np.arange(self.n,self.n+self.m),np.arange(self.n)))
        self.index =  ind
        self.T = self.x_tableau()
        self.basic = self.index[:self.m]
        self.nonbasic = self.index[self.m:]
        
    def x_tableau(self):
        A_bar = np.hstack((self.A,np.identity(self.m)))
        c_bar =  -1*np.hstack((self.c,np.zeros(self.m)))
        T_1 =  np.hstack((np.zeros(1),c_bar,np.ones(1)))
        T_2 = np.column_stack((self.b,A_bar,np.zeros((self.m,1))))
        T = np.vstack((T_1,T_2))
        return T
    
    def pivot(self):
        x =  self.T[0,:]
        col = np.argmin(np.where(x<0))+1
        if np.all(self.T[1:,col])<0:
            raise ValueError('Problem is unbounded')
        row = np.argmin(self.T[1:,0]/self.T[1:,col])+1
        return row,col
    
    def pivot_oper(self):
        row,col = self.pivot()
        ind = self.index
        ind[row-1],ind[self.m+col-1]=ind[self.m+col-1], ind[row-1]
        self.index =  ind
        self.T[row,:] = self.T[row,:]/self.T[row,col]
        dim1, dim2  = self.T.shape
        for h in range(dim1):
            if h != row:
                self.T[h,:]=self.T[h,:]-self.T[h,col]*self.T[row,:]
    
    def solve_simplex(self):
        maxiter = 1000
        iterx = 0
        while np.any(self.T[0,:] < 0) and  iterx<maxiter:
            self.pivot_oper()
            iterx += 1
        return self.T
#         basic_dict = {self.basic[i]: round(self.T[i+1, 0], 2) for i in range(self.m)}
#         nonbasic_dict = {self.nonbasic[i]: 0 for i in range(self.n)}
#         return self.T[0, 0], basic_dict, nonbasic_dict

In [5]:
c_1 = np.array([3, 2])
A_1 = np.array([[1, -1], [3, 1], [4, 3]])
b_1 = np.array([2, 5, 7])
xcv1= simplex_algo(c_1, A_1, b_1)

# xcv1.index

# m= xcv1.x_tableau()
# print(xcv1.pivot())
# print("Before",xcv1.T)
# # xcv1.pivot_oper()
pprint(xcv1.solve_simplex())



array([[ 5.        ,  0.        , -1.        ,  0.        ,  1.        ,
         0.        ,  1.        ],
       [ 0.33333333,  0.        , -1.33333333,  1.        , -0.33333333,
         0.        ,  0.        ],
       [ 1.66666667,  1.        ,  0.33333333,  0.        ,  0.33333333,
         0.        ,  0.        ],
       [ 0.33333333,  0.        ,  1.66666667,  0.        , -1.33333333,
         1.        ,  0.        ]])


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in true_divide
